In [4]:
library(repr)
library(tidyverse)
library(tidymodels)
library(dplyr)
library(RColorBrewer)
options(repr.matrix.max.rows = 10)
set.seed(1234)

options(repr.plot.width = 12, repr.plot.height = 6)

In [7]:
test_data <- read_csv("https://raw.githubusercontent.com/sophiaymeng/hack-ml/refs/heads/main/test.csv", show_col_types = FALSE)
test_data

city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,⋯,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
iq,2008,27,2008-07-01,0.3048286,0.2336714,0.2489143,0.2558143,0.00,296.5429,⋯,9.40,86.52429,0.00,15.28571,9.442857,NA,NA,NA,20.8,1.3
iq,2008,28,2008-07-08,0.1538500,0.1495857,0.3294429,0.1328857,0.00,296.5457,⋯,7.47,85.65571,0.00,15.08000,9.271429,NA,NA,NA,21.0,0.0
iq,2008,29,2008-07-15,0.2323857,0.2139571,0.2203000,0.1984286,62.84,296.0786,⋯,27.33,93.05571,62.84,16.14286,8.442857,NA,NA,NA,21.0,44.2
iq,2008,30,2008-07-22,0.3270167,0.2091167,0.2837667,0.3095500,52.36,296.5457,⋯,47.59,92.77714,52.36,16.58286,8.600000,26.60,9.775,32.4,20.8,18.1
iq,2008,31,2008-07-29,0.3548000,0.3465500,0.2674167,0.3921167,44.63,297.0443,⋯,39.00,92.04857,44.63,16.87286,9.771429,26.68,11.240,33.4,20.6,36.4
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
sj,2008,13,2008-03-25,0.07785,-0.03990000,0.31047140,0.29624290,27.19,296.9586,⋯,7.55,74.24714,27.19,13.64429,2.885714,25.04286,5.785714,30.0,21.1,1.8
sj,2008,14,2008-04-01,-0.03800,-0.01683333,0.11937140,0.06638571,3.82,298.0814,⋯,3.67,74.60000,3.82,14.66286,2.714286,26.24286,6.814286,30.6,22.2,0.5
sj,2008,15,2008-04-08,-0.15520,-0.05275000,0.13775710,0.14121430,16.96,297.4600,⋯,35.00,75.02714,16.96,14.18429,2.185714,25.00000,5.714286,29.4,21.7,30.7


In [17]:
train_data <- read_csv("https://raw.githubusercontent.com/sophiaymeng/hack-ml/refs/heads/main/train.csv") 
train_data


Rows: 1164 Columns: 25
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr   (1): city
dbl  (23): year, weekofyear, ndvi_ne, ndvi_nw, ndvi_se, ndvi_sw, precipitati...
date  (1): week_start_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,⋯,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
<chr>,<dbl>,<dbl>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
iq,2000,26,2000-07-01,0.1928857,0.1322571,0.3408857,0.2472000,25.41,296.7400,⋯,92.41857,25.41,16.65143,8.928571,26.40000,10.77500,32.5,20.7,3.0,0
iq,2000,27,2000-07-08,0.2168333,0.2761000,0.2894571,0.2416571,60.61,296.6343,⋯,93.58143,60.61,16.86286,10.314286,26.90000,11.56667,34.0,20.8,55.6,0
iq,2000,28,2000-07-15,0.1767571,0.1731286,0.2041143,0.1280143,55.52,296.4157,⋯,95.84857,55.52,17.12000,7.385714,26.80000,11.46667,33.0,20.7,38.1,0
iq,2000,29,2000-07-22,0.2277286,0.1454286,0.2542000,0.2003143,5.60,295.3571,⋯,87.23429,5.60,14.43143,9.114286,25.76667,10.53333,31.5,14.7,30.0,0
iq,2000,30,2000-07-29,0.3286429,0.3221286,0.2543714,0.3610429,62.76,296.4329,⋯,88.16143,62.76,15.44429,9.500000,26.60000,11.48000,33.3,19.1,4.0,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
sj,2004,33,2004-08-12,0.0374,0.108400,0.1253000,0.17741430,20.92,300.9957,⋯,76.17143,20.92,17.81857,2.657143,28.94286,7.228571,33.9,24.4,10.5,14
sj,2004,34,2004-08-19,-0.0164,-0.021875,0.1816000,0.17060000,37.30,300.7943,⋯,81.26429,37.30,18.77429,2.585714,29.00000,6.314286,32.2,24.4,28.3,14
sj,2004,35,2004-08-26,0.0996,0.063300,0.1006571,0.10942860,20.30,300.9129,⋯,79.68429,20.30,18.53143,2.642857,29.05714,6.485714,33.3,25.0,20.6,7


In [19]:
set.seed(1234)

# Standardization recipe
deng_recipe <- recipe(total_cases ~ ., data = train_data) |>
   step_scale(all_predictors()) |>
   step_center(all_predictors())

# KNN model specification
knn_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
            set_engine("kknn") |>
            set_mode("classification")

# 10 fold-cross valdiation
deng_vfold <- vfold_cv(train_data, v = 10, strata = city)

k_vals <- tibble(neighbors = seq(from = 1, to = 50, by = 2))

# Create workflow
knn_results <- workflow() |>
      add_recipe(deng_recipe) |>
      add_model(knn_spec) |>
      tune_grid(resamples = deng_vfold, grid = k_vals) |>
      collect_metrics() 

accuracies <- knn_results |>
      filter(.metric == "accuracy") 

best_k <- accuracies |>
        arrange(desc(mean)) |>
        head(1) |>
        select(neighbors, mean)
best_k

→ A | error:   Error in `step_scale()`:
               Caused by error in `prep()`:
               ✖ All columns selected for the step should be double or integer.
               • 1 factor variable found: `city`
               • 1 date variable found: `week_start_date`

There were issues with some computations   A: x1

Warning message:
“All models failed. Run `show_notes(.Last.tune.result)` for more information.”
There were issues with some computations   A: x10





ERROR: [1m[33mError[39m in `estimate_tune_results()`:[22m
[1m[22m[33m![39m All models failed. Run `show_notes(.Last.tune.result)` for more
  information.
